In [ ]:
# Step 1 :- Partitioning the pdf into atomic elements
# Step 2 :-

In [2]:
%pip install -Uq "unstructured[all-docs]"

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -Uq langchain_chroma

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -Uq langchain langchain_community langchain-openai
%pip install -Uq python_dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
from typing import List
from unstructured.partition.pdf import partition_pdf
from unstructured.chunking.title import chunk_by_title

from langchain_core.documents import Document
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
#Step 1 :- Partitioning the PDF
def partition_document(file_path:str):
    """Extract Elements from the PDF using unstructured"""
    print(f"Partioning document :: " + file_path)

    elements = partition_pdf(
        filename=file_path,
        strategy="hi_res",
        infer_table_structure=True,
        extract_image_block_types=["Image"],
        extract_image_block_to_payload=True
    )
    print(f"Extracted {len(elements)} elements")
    return elements


file_path = './attention_is_all_you_need.pdf'
elements = partition_document(file_path)


Partioning document :: ./attention_is_all_you_need.pdf
Extracted 173 elements


In [5]:
len(elements)

173

In [10]:
def create_chunks_by_title(elements):
    """Create intelligent chunks using title-based strategy"""
    print("🔨 Creating smart chunks...")
    
    chunks = chunk_by_title(
        elements, # The parsed PDF elements from previous step
        max_characters=3000, # Hard limit - never exceed 3000 characters per chunk
        new_after_n_chars=2400, # Try to start a new chunk after 2400 characters
        combine_text_under_n_chars=500 # Merge tiny chunks under 500 chars with neighbors
    )
    
    print(f"✅ Created {len(chunks)} chunks")
    return chunks

# Create chunks
chunks = create_chunks_by_title(elements)

🔨 Creating smart chunks...
✅ Created 21 chunks
